# 04.VAE

참고 : AE를 언제 쓰나?
- AE로 차원 축소를 해보고 그 결과를 classic한 Dim. reduction방법(t-SNE... etc.)으로 한번 더 해본다.
- 차원 축소가 잘 되었다면 manifold상에서 거리가 가깝다는 것이다. 고전적인 차원 축소 방식은 neighborhood based 에서 잘 동작하므로 manifold상에서 가까이 있다면 classic한 방법으로 차원축소를 했을때도 잘 동작할 것이기 때문이다.

### Generative Model

VAE & AE
- VAE : Generative model learning, 생성 모델, 네트워크 뒷단의 학습이 목적이다.
- AE : Manifold learning, 네트워크 앞단의 학습이 목적이다.
- 나중에 설명하겠지만 VAE와 AE는 네트워크 모양만 비슷할 뿐 수식적으로 아무 관련이 없다.

![img25](./img/img25.png)
- Generator 모델은 트레이닝 DB에서 우리가 원하는 데이터 x가 나올 확률을 구하는데 모든 트레이닝 DB에 대하여 그 확률을 최대화하는 확률 분포를 구하는 것이 목적이다.
- generator 모델에서 생성되는 데이터는 랜덤하게 나오는 거보다 컨트롤 할 수 있는것이 좋다.
    - 예를 들어 첫번째 element를 바꾸면 성별이 바뀐다던지
- z는 리모컨같은 역할을 하는 것. prior확률분포 에서 샘플링돼서 나오는 값.$z\sim p(z)$
- 네트워크 출력값 자체는 deterministic한 값. 확률분포의 parameter를 추정한다고 생각한다.$g_{\theta}(\cdot)$
- 우리가 알고 싶은것은 모든 트레이닝 DB에 대한 확률분포 $ p(x) $ 는 네트워크 추정값인 $ p(x|g_{\theta}(z))$와 미리 정해놓은 prior 확률분포 $ p(z)$를 이용해서 구한다.$$ \int p(x|g_{\theta}(z))p(z) dz = p(x)$$ 
- z를 어떻게 Generator에 넣는냐에 따라서 출력값이 달라지므로 $p(z)$우리가 다루기 쉬운 확률분포를 사용한다. normal or uniform distribution
- 그런데 $p(z)$를 단순한 분포를 사용해도 상관이 없나??
    - z는 결국 latent space(manifold) 상에 있는 값을 취하는것. 복잡할 것 같은 공간인데 단순한 확률분포를 사용하는 것이 그 공간을 잘 설명하는 것이 맞는가??
    - 어차피 Deep NN에서 앞쪽에 레이어 1~2개가 복잡한 latent space를 익히므로 prior distribution은 단순한 걸 써도 괜찮다.
    - ref)https://arxiv.org/pdf/1606.05908.pdf figure2
    - 단순한 분포에서 z를 뽑아도 이후에 약간의 테크닉만으로도 완전히 다른 분포를 만들어낼 수 있다. 

- $p(x)$를 구하기 위한 값을 모두 알고 있는 값인데 왜 바로 MLE하지 않는가? 샘플별로 구해서 summation하면 구할 수 있지 않은가?
-  안된다!! VAE로 해야한다.
![img26](./img/img26.png)
- 위 그림에서 (a)는 원 샘플, (b)는 일부분을 지운것, (c)는 (a)에서 화소값을 왼쪽으로 1씩 쉬프트한것이다. 의미적으로 봤을때 (a)와 (c)가 더 가까워 보인다.
- 하지만 MSE를 구해보면 (a)와 (b)의 MSE가 더 작게 나온다. 모델을 MSE가 작은쪽으로 학습하므로 이렇게 되면 학습된 모델에서 샘플링했을때 (c)보다 (b)가 나올 확률이 높아진다.(우리가 원하는 건 의미적으로 더 가까운 (c)인데)
- 즉, MSE가 작다고 해서 의미적으로 가까운것이 아니라는것. **Sampling 함수를 잘 정해야한다.**
    - 어떤게 이상적인 Sampling함수인지 모르므로 추정해야 한다. Variation Inference를 통해서!

### Variational Inference

이상적인 Sampling함수는 결국 $x$와 유사한 샘플이 나올수 있는 확률분포 $p(z|x)$ 인데 무엇인지 알 수 없으므로 우리가 알고있는 확률분포 $ q_{\phi}(z|x)$의 parameter를 바꿔가면서 $p(z|x)$와 유사하게 만들어간다.(Approximation)